In [1]:
using DataFrames, Distributions, DataFramesMeta

In [2]:
function gen_data(; n = 500000, mu = [0, 2], sigma = 1, pZ1 = 0.8)
    data = DataFrame(
        Obs_ID = 1:n,
        Z = (rand(Uniform(0,1), n) .> pZ1) .+ 1
        )

    @transform!(data, :Y = rand(Normal(0, sigma), n) + mu[:Z])    
    @select!(data, :Obs_ID, :Y)
    @transform!(data, :p_Y_given_Z_1 = pdf.(Normal(mu[1], sigma), :Y))
    @transform!(data, :p_Y_given_Z_2 = pdf.(Normal(mu[2], sigma), :Y))

    return data
  end

gen_data (generic function with 1 method)

In [3]:
data = gen_data(n = 500000, pZ1 = 0.8);
first(data, 6)

,Obs_ID,Y,p_Y_given_Z_1,p_Y_given_Z_2
,Int64,Float64,Float64,Float64
1,1,0.94735,0.254699,0.229242
2,2,-0.720732,0.307689,0.0098515
3,3,-0.135681,0.395287,0.0407823
4,4,2.87999,0.00630687,0.270866
5,5,-0.960182,0.2516,0.00499021
6,6,-0.0763223,0.397782,0.0462129


In [4]:
function fit_model!(
    data; 
    pi_hat_0 = 0.5, 
    tolerance = 0.0001,
    max_iterations = 1000,
    progress = DataFrame(
        iter = 1:(max_iterations+1), 
        pi_hat = Vector{Float64}(undef, max_iterations+1), 
        ll = Vector{Float64}(undef, max_iterations+1), 
        ll_diff = Vector{Float64}(undef, max_iterations+1)
        )
    )

    pi_hat = pi_hat_0
    E_step!(data, pi_hat)
    ll = loglik!(data)
    # progress = [(iter = 0, pi_hat, ll, ll_diff = NaN)]
    progress[1,:] = (0, pi_hat, ll, NaN)
    
    last_iter = 0
    for i in 1:max_iterations
        pi_hat = M_step(data)
        E_step!(data, pi_hat)
        
        ll_old = ll
        ll = loglik!(data)
        ll_diff = ll - ll_old
        progress[i+1,:] = (i, pi_hat, ll, ll_diff)

        if ll_diff < tolerance
            last_iter = i
            break
        end
    end
    return progress[1:last_iter, :]
end

fit_model! (generic function with 1 method)

In [5]:
function E_step!(data, pi_hat)
    @transform!(data, :pY_Z1 = :p_Y_given_Z_1 .* pi_hat)
    @transform!(data, :pY_Z2 = :p_Y_given_Z_2 .* (1- pi_hat))
    @transform!(data, :pY = :pY_Z1 + :pY_Z2)
    @transform!(data, :pZ1_given_Y = :pY_Z1 ./ :pY)
end

E_step! (generic function with 1 method)

In [6]:
function M_step(data)
    mean(data[!, :pZ1_given_Y])
end

M_step (generic function with 1 method)

In [7]:
function loglik!(data)
    sum(log.(data[!, :pY]))
end

loglik! (generic function with 1 method)

In [8]:
@time progress = fit_model!(data, tolerance = .00001)


  0.814848 seconds (3.18 M allocations: 533.181 MiB, 7.65% gc time, 82.25% compilation time)


,iter,pi_hat,ll,ll_diff
,Int64,Float64,Float64,Float64
1,0,0.5,-8.77138e5,NaN
2,1,0.664971,-8.36746e5,40392.0
3,2,0.737143,-8.27814e5,8931.34
4,3,0.769548,-8.25747e5,2067.51
5,4,0.784872,-8.25244e5,503.043
6,5,0.792374,-8.25117e5,126.271
7,6,0.796119,-8.25085e5,32.2466
8,7,0.798008,-8.25077e5,8.31041
9,8,0.798967,-8.25075e5,2.15187
